# Наша n-ая свёрточная нейросеть 

Пришло время построить нашу первую свёрточную нейросеть. Будем использовать для этого датасет [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) (Canadian Institute for Advanced Research). Он включает в себя картинки из 10 разных классов: самолёты, машины, птицы, кошки, олени, собаки, лягушки, лошади, корабли, грузовики.   

<img src="http://www.pvsm.ru/images/2016/11/18/optimizaciya-neirosetevoi-platformy-Caffe-dlya-arhitektury-Intel-3.png" style="width:50%">

Всего $60 000$ цветных картинок размера $32 \times 32$. В каждом из классов ровно по $6000$ картинок.  Есть расширение этого датасета, CIFAR_100. Думаю, что по названию вы догадались, что в нём сто видов картинок. Попробовать обуздать этот датасет можно, подгрузив его следущими командами: 

```from keras.datasets import cifar100
   (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')```

In [ ]:
# подгружаем пакеты
import numpy as np
import random
from tqdm import tqdm

import tensorflow as tf
import keras
from keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

## 1. Скачиваем и готовим данные

Приготовьте на своём компухтере местечко для датасета. Он достаточно громоздкий и включает 60 000 картинок. Кстати говоря, если очень хочется, можете принять участие [в стареньком соревновании на Kagle,](https://www.kaggle.com/c/cifar-10) связанным с этим датасетом. Там же на форуме можно найти интересный код. :) 

In [ ]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print("Train samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

Помним, что у нас всего 10 классов.

In [ ]:
NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

Нарисуем несколько рандомных картинок из тренировочной выборки. 

In [ ]:
# ваш код здесь 

Отлично! Как вы помните, если пронормаровать данные, то сетка будет сходиться на порядок быстрее. 

Также, как вы помните из предыдущих скриптов, картинка - это тензор из циферок. Каждая циферка сообщает нам о яркости конкретного пикселя. Яркость измеряется по шкале от 0 до 255. В связи с этим фактом, нормализация будет немного странной: 

$$
x_{norm} = \frac{x}{255}
$$

Также мы помним, что классы нужно конвертировать одним горячи кодированием (one-hot encoding) в набор из дамми-переменных. 

In [ ]:
# делай раз

# ващ код 

# делай два! 

# ваш код 

## 2.  Выбираем для нашей нейросети архитектуру

Свёрточная нейронная сеть строится из нескольких разных типов слоёв: 

* [Conv2D](https://keras.io/layers/convolutional/#conv2d) - Конволюция:
    - **filters**: число выходных каналов; 
    - **kernel_size**: размер окна для свёртки;
    - **padding**: padding="same" добавляет нулевую каёмку по краям картинки, чтбы после свёртки размеры картинки не изменялись; padding='valid' ничего не добавляет;
    - **activation**: "relu", "tanh", etc.
    - **input_shape**: размер входа
* [MaxPooling2D](https://keras.io/layers/pooling/#maxpooling2d) - макспулинг
* [Flatten](https://keras.io/layers/core/#flatten) - разворачивает картинку в вектор 
* [Dense](https://keras.io/layers/core/#dense) - полносвязный слой (fully-connected layer)
* [Activation](https://keras.io/layers/core/#activation) - функция активации
* [LeakyReLU](https://keras.io/layers/advanced-activations/#leakyrelu) - leaky relu активация
* [Dropout](https://keras.io/layers/core/#dropout) - дропаут.


В модели, которую мы определим ниже, на вход будет идти тензоры размера __(None, 32, 32, 3)__ и __(None, 10)__. На выходе мы будем получать вероятноть того, что объект относится к конкретному классу. Разменость __None__ заготовлена для размерности батча. 

In [ ]:
# подгружаем важные строительные блоки
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, InputLayer
from keras.layers.advanced_activations import LeakyReLU

Соберите сетку со следующей архитектурой: 

* Четыре свёрточных слоя с ядром $3 \times 3$ и числом фильтров $16, 32, 32, 64$. Используйте same padding. 
* Слой пулинга размера $2 \times 2$. после каждыйх двух свёрточных слоёв. 
* В качестве функции активации используйте LeakyReLU с параметром $0.1$. Используйте её после каждого слоя. 
* Разверните сеть в полносвязную, добавьте слой с $256$ нейронами.
* На выходе используйте __Softmax__
* После полносвязного слоя добавьте __Dropout__ с вероятностью $0.5$, после каждого макспулинга добавьте __Dropout__ с вероятностью $0.25$. 

In [ ]:
def make_model():
    """
    Определите архитектуру свой сетки внутри этой функции
    """

    # ваш код 
    
    
    return model

In [ ]:
# Взглянем на нашу модель
model = make_model()
model.summary()

Огромное количество параметров нам предстоит оценить. 

## 3. Оцениваем модель

Обучение модели может занять примерно 4-8 минут на каждой эпохе. В случае, если на картинке качество модели не будет расти, придётся переопределять параметры вроде скорости обучения. Если и это не поможет, придётся думать о новой архитектуре. 

In [ ]:
INIT_LR = 5e-3    # Скорость обучения 
BATCH_SIZE = 32   # Размер батча
EPOCHS = 10       # Эпохи 

model = make_model()  # Создаём модель 

# Выбираем для модели оптимизатор и собираем её
model.compile(
    loss='categorical_crossentropy',  # Кросс энтропия обычно используется как функция потерь для 
                                      # задачи многоклассовой классификации
    optimizer=keras.optimizers.adamax(lr=INIT_LR),  # SGD наш оптимизатор 
    metrics=['accuracy']  # Будем в ходе обучения запоминать точность прогнозов
)

# Новая фишка! Функция, которая корректирует скорость обучения каждую эпоху 
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# Мы готовы к обучению 
history = model.fit(
    x_train2, y_train2,    # Данные
    batch_size=BATCH_SIZE, # Сколько сетка кушает за раз 
    epochs=EPOCHS,         # Сколько полных проходов по данным
    # Колбэки. Через них реализован всякий полезный функционал. В прошлый раз мы 
    # с помощью этого параметра делали early stopping 
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler)],
    validation_data=(x_test2, y_test2),  # Немножко валидации :) 
    shuffle=True
)

In [ ]:
# Сохраняем веса модели в файл ибо модель училась долго 
model.save_weights("cifar_10_weights.h5")

In [ ]:
# Можем подгрузить веса назад, если случилась беда и комп отрубило 
model = make_model()
model.load_weights("cifar_10_weights.h5")

Построим как проходил процесс обучения и какого качества нам удалось добиться. 

In [ ]:
start = 0
plt.plot(history.history['loss'][start:])
plt.plot(history.history['val_loss'][start:])
plt.legend(['Train loss', 'Validation loss'])

In [ ]:
y_pred_test = model.predict_proba(x_test2)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
y_pred_test_max_probas = np.max(y_pred_test, axis=1)

In [ ]:
# confusion matrix and accuracy
from sklearn.metrics import confusion_matrix, accuracy_score
plt.figure(figsize=(7, 6))
plt.title('Confusion matrix', fontsize=16)
plt.imshow(confusion_matrix(y_test, y_pred_test_classes))
plt.xticks(np.arange(10), cifar10_classes, rotation=45, fontsize=12)
plt.yticks(np.arange(10), cifar10_classes, fontsize=12)
plt.colorbar()
plt.show()
print("Test accuracy:", accuracy_score(y_test, y_pred_test_classes))

Хммм... 80% это неплохо. Судя по матрице, модель не совершает каких-то систематических ошибок в одном из конкретных классов. Например, она не путает лошадей с оленями. Попробуем сделать пару предсказаний и посмотреть на картинки. 

In [ ]:
cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 3 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(y_test))    # Выбираем рандомный объект из теста 
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)  # выделяем место для картинки 
        ax.grid('off')                                      # октлючаем решётку 
        ax.axis('off')                                      # отключаем оси 
        ax.imshow(x_test[random_index, :])                  # рисуем картинку 
        pred_label = cifar10_classes[y_pred_test_classes[random_index]]  # истиный класс
        pred_proba = y_pred_test_max_probas[random_index]                # вероятность 
        true_label = cifar10_classes[y_test[random_index, 0]]            # предсказанный класс
        ax.set_title("pred: {}\nscore: {:.3}\ntrue: {}".format(
               pred_label, pred_proba, true_label    # Подписываем картинки 
        ))
plt.show()

Теперь попробуйте добавить в сетку выше `batch_norm` слой с дефолтными параметрами между каждой свёрткой и пулингом. Переобучите сетку. Прокомментируйте произошедшие изменения: 

**Комментарий:**


## 4. Data augmentation

Попробуем обучить ту же модель, но искуственно расширяя набор данных за счёт [случайных искажений.](https://machinelearningmastery.com/image-augmentation-deep-learning-keras/)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

* `rotation_range`  значение в градусах (0-180), диапазон, в пределах которого произвольно вращаются изображения;
* `width_shift` и `height_shift` это диапазоны (в долях от общей ширины или высоты), в пределах которых можно произвольно переводить изображения по вертикали или горизонтали;
* `rescale` это коэффициент скалирования, на который мы умножаем наши данные перед каждой модернизацией;
* `shear_range` диапазон для рандомных сдвигов
* `zoom_range` для случайного масштабирования внутри фотографий
* `horizontal_flip` для переворачивания половины изображения по горизонтали
* `fill_mode` стратегия для заполнения вновь появившихся пикселей

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=255,
        shear_range=0,
        zoom_range=0.1,
        horizontal_flip=True)

In [ ]:
datagen.fit(x_train2)  # зафитим генератор 

In [ ]:
it = datagen.flow(x_train2, y_train2, batch_size=15) # итератор по данным из выборки

images, categories = it.next()
print("Number of images returned by iterator:", len(images))

# преобразуем часть картинок и посмотрим как они теперь выглядят 
fig = plt.figure(figsize=(15, 10))
for i in range(15):
    plt.subplot(3, 5, i+1)
    im = images[i]
    c = np.where(categories[i] == 1)[0][0] # convert one-hot to regular index
    plt.imshow(im)
    plt.axis('off')

In [ ]:
# посмотрим как наша аугментация сказывается на какой-то одной конкретной картинке 
it = datagen.flow(np.array(15*[x_train2[200]]),np.array(15*[y_train2[200]]), batch_size=15) # итератор 

images, categories = it.next()
print("Number of images returned by iterator:", len(images))

fig = plt.figure(figsize=(15, 10))
for i in range(15):
    plt.subplot(3, 5, i+1)
    im = images[i]
    c = np.where(categories[i] == 1)[0][0] # convert one-hot to regular index
    plt.imshow(im)
    plt.axis('off')

Собираем вместе с нашим генератором нейронку. 

In [ ]:
model =  make_model()
  
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

datagen.fit(x_train2)

In [ ]:
# учим модель на сгенерированных батчах 
dgen = imdgen.flow(x_train2, y_train2, batch_size=32)

# при каждой эпохе картинка искажается по-новому и попадает на вход в нашу сетку
hist = model.fit_generator(
    dgen,
    samples_per_epoch = x_train2.shape[0],
    nb_epoch = 5,
    validation_data=(x_test2, y_test2),
    verbose = 1
)

In [ ]:
# Посмотрите насколько сильно удалось улучшить качество модели

# Ваш код 

Попытайтесь максимально повысить качество своей нейросетки на тестовой выборке. Можете использовать для этого любые приёмы и слои. Все свои идеи и результат их имплементации кратко изложите ниже: 

**Описание ваших экспериментов:**